<a href="https://colab.research.google.com/github/dharris50/Test/blob/master/compression_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title
#%%


import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#%%
import tensorflow as tf

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import math


#used to fix bug in keras preprocessing scope
temp = tf.zeros([4, 32, 32, 3])  # Or tf.zeros
preprocess_input(temp)
print("processed")


#%%

# IMAGE_SIZE = (224, 224)
# TRAIN_SIZE = 50000
# VALIDATION_SIZE = 10000
# BATCH_SIZE_PER_GPU = 64
# global_batch_size = (BATCH_SIZE_PER_GPU * 1)
# NUM_CLASSES = 10
# TEST = 1
# EPOCHS = 20 if TEST == 1 else 2

# for command line arguments - sys.argv[1] contains the config file
import sys

# for reading the config file
import configparser
config = configparser.ConfigParser()
config.read('variables.ini')

# set variables
IMAGE_SIZE = (int(config['Image Size']['width']), int(config['Image Size']['length']))
TRAIN_SIZE = int(config['Other']['train size'])
VALIDATION_SIZE = int(config['Other']['validation size'])
BATCH_SIZE_PER_GPU = int(config['Other']['batch size per gpu'])
global_batch_size = (BATCH_SIZE_PER_GPU * 1)
NUM_CLASSES = int(config['Other']['number of classes'])
EPOCHS = int(config['Other']['epochs'])
NUM_PROC = int(config['Other']['num_proc'])

# import functions from util file
from utils_modified.py import *

#%% 
# Dataset code

#%%


dataset, info = tfds.load('cifar10', with_info=True)


train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train.shuffle(buffer_size=1000).batch(global_batch_size).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


test_dataset = dataset['test'].map(load_image_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(global_batch_size).repeat()
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


model = tf.keras.models.load_model('./base_model_cifar10_vgg16.h5')
model.compile(optimizer=tf.optimizers.SGD(learning_rate=.01, momentum=.9, nesterov=True), loss='mse', metrics=['acc'])
OG = model.evaluate(test_dataset, steps=VALIDATION_SIZE//global_batch_size//TEST)
print(OG)


#%%



#%%
import pprint
targets = []
for i, layer in enumerate(model.layers):
    if layer.__class__.__name__ == "Conv2D":
        if layer.kernel_size[0] == 3:
            #print(f'{i} layer {layer.name} , kernel size {layer.kernel_size}')
            targets.append({'name': layer.name, 'layer': i})

pprint.pprint(targets)



for target in targets:

    writer = tf.summary.create_file_writer(f"./summarys/vgg/cifar10/{target['name']}")
    with writer.as_default():
        print(f"training layer {target['name']}")
        tf.keras.backend.clear_session()
        model = tf.keras.models.load_model('base_model_cifar10_vgg16.h5')
        in_layer = target['layer']
        get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[in_layer - 1].output,
                                                                model.layers[in_layer].output])


        replacement_layers = build_replacement(get_output, layers=2)
        replacement_len = len(replacement_layers.layers)
        layer_train_gen = LayerBatch(get_output, train_dataset)
        layer_test_gen = LayerTest(get_output, test_dataset)




        MSE = tf.losses.MeanSquaredError()

        optimizer=tf.keras.optimizers.SGD(.00001, momentum=.9, nesterov=True)
        replacement_layers.compile(loss=MSE, optimizer=optimizer)

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=5, min_lr=.0001, factor=.3, verbose=1)
        early_stop = tf.keras.callbacks.EarlyStopping(patience=15, min_delta=.0001, restore_best_weights=True, verbose=1)

        replacement_layers.save('/tmp/layer.h5')

        for epoch in range(EPOCHS + 1):

            tf.keras.backend.clear_session()
            model = tf.keras.models.load_model('base_model_cifar10_vgg16.h5')
            in_layer = target['layer']
            get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[in_layer - 1].output,
                                                                    model.layers[in_layer].output])



            layer_train_gen = LayerBatch(get_output, train_dataset)
            layer_test_gen = LayerTest(get_output, test_dataset)

            replacement_layers = tf.keras.models.load_model('/tmp/layer.h5')

            history = replacement_layers.fit(x=layer_train_gen,
                                        epochs=1,
                                        steps_per_epoch=TRAIN_SIZE // global_batch_size // TEST,
                                        validation_data=layer_test_gen,
                                        shuffle=False,
                                        callbacks=[reduce_lr, early_stop],
                                        validation_steps=VALIDATION_SIZE // global_batch_size // TEST,
                                        verbose=1)

            replacement_layers.save('/tmp/layer.h5')

            target['weights'] = [replacement_layers.layers[1].get_weights(), replacement_layers.layers[3].get_weights()]

            tf.keras.backend.clear_session()

            model = tf.keras.models.load_model('base_model_cifar10_vgg16.h5')
            layer_name = target['name']
            layer_pos = target['layer']
            filters = model.layers[layer_pos].output.shape[-1]

            new_model = replace_layer(model, layer_name, lambda x: replac(x, filters))
            new_model.layers[layer_pos].set_weights(target['weights'][0])
            new_model.layers[layer_pos + 2].set_weights(target['weights'][1])
            new_model.compile(optimizer=tf.keras.optimizers.SGD(.1), loss="categorical_crossentropy", metrics=['accuracy'])
            target['score'] = new_model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size // TEST)

            tf.summary.scalar(name='rep_loss', data=history.history['loss'][0], step=epoch)
            tf.summary.scalar(name='val_loss', data=history.history['val_loss'][0], step=epoch)
            tf.summary.scalar(name='model_acc', data=target['score'][1], step=epoch)
            tf.summary.scalar(name='model_loss', data=target['score'][0], step=epoch)

            writer.flush()
            print(f"epoch: {epoch}, rep loss {history.history['loss']}, val loss {history.history['val_loss']}, model acc {target['score'][1]}")

            # if np.abs(OG[1] - target['score'][1]) < 0.0001:
            #     print('stoping early')
            #     break





tf.keras.backend.clear_session()
model = tf.keras.models.load_model('./base_model_cifar10_vgg16.h5')


writer = tf.summary.create_file_writer(f"./summarys/vgg/cifar10/final_model")
with writer.as_default():
    for target in targets[::-1]:
        print(f'replacing layer {target["name"]}')

        layer_name = target['name']
        layer_pos = target['layer']
        filters = model.layers[layer_pos].output.shape[-1]



        new_model = replace_layer(model, layer_name, lambda x: replac(x, filters))
        new_model.layers[layer_pos].set_weights(target['weights'][0])
        new_model.layers[layer_pos + 2].set_weights(target['weights'][1])

        new_model.save('cifar10_vgg_modified.h5')
        tf.keras.backend.clear_session()
        model = tf.keras.models.load_model('cifar10_vgg_modified.h5')

    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model('cifar10_vgg_modified.h5')
    model.compile(optimizer=tf.keras.optimizers.SGD(.1), loss="categorical_crossentropy", metrics=['accuracy'])
    final = model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size // TEST)

    tf.summary.scalar(name='model_acc', data=final[1], step=0)
    tf.summary.scalar(name='model_loss', data=final[0], step=0)

    writer.flush()


#%%
# for target in targets:

#     print(f'Replacing Layer {target["name"]}')

#     tf.keras.backend.clear_session()

#     model = tf.keras.models.load_model('base_model_cifar10_vgg16.h5')

#     layer_name = target['name']
#     layer_pos = target['layer']
#     filters = model.layers[layer_pos].output.shape[-1]


#     new_model = replace_layer(model, layer_name, lambda x: replac(x, filters))
#     new_model.layers[layer_pos].set_weights(target['weights'][0])
#     new_model.layers[layer_pos + 2].set_weights(target['weights'][1])
#     new_model.compile(optimizer=tf.keras.optimizers.SGD(.1), loss="categorical_crossentropy", metrics=['accuracy'])
#     target['score'] = new_model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size)


AssertionError: ignored